# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

A partir del dataset elegido y ya preprocesado con la lógica definida en el TP1, el grupo debe realizar el entrenamiento y evaluación de al menos 3 algoritmos de machine learning.

   Se debe elegir y definir una métrica de performance a utilizar para evaluar los modelos. Fundamentar la elección de la métrica.
   Se debe aplicar alguna técnica de feature engineering para mejorar los datos de entrada a los modelos, y mostrar la comparativa de los resultados obtenidos en cada caso. Si no es posible o útil, fundamentar el motivo por el cual no se realizará.
   Por cada modelo, se debe entrenarlo y realizar una exploración de hiper-parámetros mediante una búsqueda en grilla. Evaluar el comportamiento de cada modelo con los hiper-parámetros que mejores resultados ofrecen. En caso de ser posible, aporte conclusiones respecto a dicha comparación.
   Realizar experimentos que utilicen como datos de entrada representaciones intermedias de los datos (generadas por técnicas de reducción de dimensiones como PCA). Compare los resultados obtenidos contra los casos previos, interprete y proponga conclusiones.
   Se deben utilizar técnicas que garanticen que los modelos no están sobreentrenando sin que nos demos cuenta.
   Determinar el valor final de la métrica que podría ser informado al cliente, utilizando técnicas que permitan obtener un valor lo más realista posible. Fundamentar y considerar no solo el rendimiento del modelo en su elección, sino también cuestiones como interpretabilidad, tiempos de entrenamiento, etc.
   Para el método propuesto como definitivo, y para distintos pares de variables, genere diagramas de dispersión donde se visualicen los aciertos y errores del mismo. Discuta si existen patrones o conocimiento que se pueda obtener a partir de dichos errores. En caso de ser posible, evalúe la importancia que asigna el método a las variables de entrada y genere conclusiones al respecto.


Tratamiento de datos aplicando en el TPN°1

In [ ]:
# Importamos las dependencias necesarias.
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import seaborn as sns

In [ ]:
# Importamos train.csv y park_dimensions.csv, los unimos utilizando la variable "park"
entrenamiento = pd.read_csv('./train.csv')
estadio = pd.read_csv('./park_dimensions.csv')
completa=entrenamiento.merge(estadio, on="park", how="left")

# Desechamos las variables no utilizadas
completa = completa.drop(['park','bip_id','batter_id','pitcher_id'],axis=1)

In [ ]:
# Asignamos nuevos nombres a las columnas
renamed_columns = {'NAME': 'name', 'Cover': 'cover', 'LF_Dim': 'lf_dim', 'CF_Dim':'cf_dim',
                   'RF_Dim': 'rf_dim', 'LF_W': 'lf_w', 'CF_W': 'cf_w', 'RF_W': 'rf_w'
                  }
completa.rename(columns=renamed_columns, inplace=True)

# Convertir columna "game_date" de tipo object/string, a datetime
completa['game_date'] = pd.to_datetime(completa['game_date'])

In [ ]:
completa[['inning', 'outs_when_up', 'balls', 'strikes', 'plate_x', 
           'plate_z', 'pitch_mph', 'launch_speed', 'launch_angle', 
           'lf_dim', 'cf_dim', 'rf_dim', 'lf_w', 'cf_w', 'rf_w']
         ].describe(include='all')